In [1]:
install.packages("dplyr")
install.packages("RSQLite")
install.packages("sqldf")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library(RSQLite)
library(dplyr)
library(sqldf)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: gsubfn
Loading required package: proto


In [3]:
# read the database file

dbfile<-"/Users/mitalibhiwande/Desktop/DIC/database.sqlite"

In [4]:
#Cinnect to database
dbcon <- dbConnect(dbDriver("SQLite"),dbname = dbfile)

In [5]:
alltables = dbListTables(dbcon)

In [6]:
print(alltables)

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"  


#### Read all the tables as dataframes

In [7]:
player.df = dbGetQuery(dbcon,'select * from Player' )

In [8]:
team.df = dbGetQuery(dbcon,'select * from Team' )

In [9]:
playerattri.df = dbGetQuery(dbcon,'select * from Player_Attributes' )

In [10]:
teamattri.df = dbGetQuery(dbcon,'select * from Team_Attributes' )

### Perform data munging on player, player attribute and match table to obtain players.csv

In [11]:
pa.df<-subset(playerattri.df, select = c(player_api_id,date,overall_rating,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes))

In [12]:
head(pa.df)

player_api_id,date,overall_rating,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
505942,2016-02-18 00:00:00,67,6,11,10,8,8
505942,2015-11-19 00:00:00,67,6,11,10,8,8
505942,2015-09-21 00:00:00,62,6,11,10,8,8
505942,2015-03-20 00:00:00,61,5,10,9,7,7
505942,2007-02-22 00:00:00,61,5,10,9,7,7
155782,2016-04-21 00:00:00,74,14,7,9,9,12


In [13]:
pa.df$gk_rating <- pa.df$gk_diving + pa.df$gk_handling + pa.df$gk_kicking + pa.df$gk_positioning + pa.df$gk_reflexes

In [14]:
head(pa.df)

player_api_id,date,overall_rating,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_rating
505942,2016-02-18 00:00:00,67,6,11,10,8,8,43
505942,2015-11-19 00:00:00,67,6,11,10,8,8,43
505942,2015-09-21 00:00:00,62,6,11,10,8,8,43
505942,2015-03-20 00:00:00,61,5,10,9,7,7,38
505942,2007-02-22 00:00:00,61,5,10,9,7,7,38
155782,2016-04-21 00:00:00,74,14,7,9,9,12,51


In [15]:
pa1.df<-subset(pa.df, select=c(player_api_id,date,overall_rating,gk_rating))

In [16]:
head(pa1.df)

player_api_id,date,overall_rating,gk_rating
505942,2016-02-18 00:00:00,67,43
505942,2015-11-19 00:00:00,67,43
505942,2015-09-21 00:00:00,62,43
505942,2015-03-20 00:00:00,61,38
505942,2007-02-22 00:00:00,61,38
155782,2016-04-21 00:00:00,74,51


In [17]:
player1.df<-subset(player.df,select=c(player_api_id,player_name))

In [18]:
head(player1.df)

player_api_id,player_name
505942,Aaron Appindangoye
155782,Aaron Cresswell
162549,Aaron Doran
30572,Aaron Galindo
23780,Aaron Hughes
27316,Aaron Hunt


In [19]:
playermerge=merge(pa1.df,player1.df,by="player_api_id")

In [20]:
head(playermerge)

player_api_id,date,overall_rating,gk_rating,player_name
2625,2015-01-16 00:00:00,61,45,"Patryk Rachwal,18"
2625,2014-11-07 00:00:00,61,45,"Patryk Rachwal,18"
2625,2014-09-18 00:00:00,61,45,"Patryk Rachwal,18"
2625,2013-06-07 00:00:00,61,45,"Patryk Rachwal,18"
2625,2013-03-08 00:00:00,61,45,"Patryk Rachwal,18"
2625,2013-02-15 00:00:00,58,45,"Patryk Rachwal,18"


In [21]:
match.df<-dbGetQuery(dbcon,'select * from Match' )

In [22]:
m.df<-match.df[,56:77]

In [23]:
head(m.df)

home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,⋯,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [24]:
m.df<-na.omit(m.df)

In [25]:
head(m.df)

,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,⋯,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
146,38327,67950,67958,67959,37112,36393,148286,67898,164352,38801,⋯,38293,148313,104411,148314,37202,43158,9307,42153,32690,38782
154,36835,37047,37021,37051,104386,32863,37957,37909,38357,37065,⋯,21812,11736,37858,38366,37983,39578,38336,52280,27423,38440
156,34480,38388,26458,13423,38389,30949,38393,38253,38383,38778,⋯,37886,37903,37889,94030,37893,37981,131531,130027,38231,131530
163,38327,67950,67958,38801,67898,37112,67959,148286,164352,33657,⋯,38388,38389,31316,164694,30949,38378,38383,38393,38253,37069
169,37900,37886,37100,37903,37889,37893,37981,131531,131530,38231,⋯,38247,16387,94288,94284,45832,26669,33671,163670,37945,33622
174,104378,38248,36841,38255,104382,33662,37044,32760,38229,39875,⋯,37887,37886,37903,94030,37981,131531,131530,130027,38231,75500


In [26]:
sumcount<-as.data.frame(table(c(m.df$home_player_1,m.df$home_player_2,m.df$home_player_3,m.df$home_player_4,m.df$home_player_5,m.df$home_player_6,m.df$home_player_7,m.df$home_player_8,m.df$home_player_9,m.df$home_player_10,m.df$home_player_11,m.df$away_player_1,m.df$away_player_2,m.df$away_player_3,m.df$away_player_4,m.df$away_player_5,m.df$away_player_6,m.df$away_player_7,m.df$away_player_8,m.df$away_player_9,m.df$away_player_10,m.df$away_player_11)))

In [27]:
head(sumcount)

Var1,Freq
2625,4
2752,79
2768,69
2770,14
2790,40
2796,1


In [28]:
playermerge<-merge(x=playermerge, y=sumcount, by.x="player_api_id", by.y="Var1")

In [29]:
head(playermerge)

player_api_id,date,overall_rating,gk_rating,player_name,Freq
2625,2007-02-22 00:00:00,63,111,"Patryk Rachwal,18",4
2625,2007-08-30 00:00:00,63,153,"Patryk Rachwal,18",4
2625,2008-08-30 00:00:00,60,153,"Patryk Rachwal,18",4
2625,2012-02-22 00:00:00,58,45,"Patryk Rachwal,18",4
2625,2011-08-30 00:00:00,58,45,"Patryk Rachwal,18",4
2625,2014-09-18 00:00:00,61,45,"Patryk Rachwal,18",4


In [30]:
names(playermerge)[names(playermerge) == 'Freq'] <- 'NumofAppearances'

In [31]:
head(playermerge)

player_api_id,date,overall_rating,gk_rating,player_name,NumofAppearances
2625,2007-02-22 00:00:00,63,111,"Patryk Rachwal,18",4
2625,2007-08-30 00:00:00,63,153,"Patryk Rachwal,18",4
2625,2008-08-30 00:00:00,60,153,"Patryk Rachwal,18",4
2625,2012-02-22 00:00:00,58,45,"Patryk Rachwal,18",4
2625,2011-08-30 00:00:00,58,45,"Patryk Rachwal,18",4
2625,2014-09-18 00:00:00,61,45,"Patryk Rachwal,18",4


In [32]:
joining<- dbGetQuery(dbcon, "SELECT count(distinct((league_id))) as numberOfLeagues, player_api_id FROM (Player INNER JOIN Match) WHERE (player_api_id=home_player_1 OR player_api_id=home_player_2 OR player_api_id=home_player_3 OR player_api_id=home_player_4 OR player_api_id= home_player_5 OR player_api_id= home_player_6 OR player_api_id= home_player_7 OR player_api_id=home_player_8 OR player_api_id=home_player_9 OR player_api_id=home_player_10 OR player_api_id=home_player_11 OR player_api_id=away_player_1 OR player_api_id=away_player_2 OR player_api_id=away_player_3 OR player_api_id=away_player_4 OR player_api_id=away_player_5 OR player_api_id=away_player_6 OR player_api_id=away_player_7 OR player_api_id=away_player_8 OR player_api_id=away_player_9 OR player_api_id=away_player_10 OR player_api_id=away_player_11) GROUP BY player_api_id")

In [33]:
playermerge<-merge(x=playermerge, y=joining, by.x="player_api_id", by.y="player_api_id")

## Final desired Player table

In [34]:
head(playermerge)

player_api_id,date,overall_rating,gk_rating,player_name,NumofAppearances,numberOfLeagues
2625,2007-02-22 00:00:00,63,111,"Patryk Rachwal,18",4,1
2625,2007-08-30 00:00:00,63,153,"Patryk Rachwal,18",4,1
2625,2008-08-30 00:00:00,60,153,"Patryk Rachwal,18",4,1
2625,2012-02-22 00:00:00,58,45,"Patryk Rachwal,18",4,1
2625,2011-08-30 00:00:00,58,45,"Patryk Rachwal,18",4,1
2625,2014-09-18 00:00:00,61,45,"Patryk Rachwal,18",4,1


### Perform data munging on team, team attribute and match table to obtain teams.csv

In [35]:
teammerge<-dbGetQuery(dbcon, "SELECT team_api_id, team_long_name, date, (buildUpPlaySpeed+ buildUpPlayDribbling+ buildUpPlayPassing) AS build_up_play, (chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting) AS chance_creation, ( defencePressure+defenceAggression+defenceTeamWidth ) AS defence FROM Team_Attributes INNER JOIN Team USING(team_api_id)")


In [36]:
head(teammerge)

team_api_id,team_long_name,date,build_up_play,chance_creation,defence
9930,FC Aarau,2010-02-22 00:00:00,NA,180,150
9930,FC Aarau,2014-09-19 00:00:00,156,181,145
9930,FC Aarau,2015-09-10 00:00:00,142,181,145
8485,Aberdeen,2010-02-22 00:00:00,NA,210,200
8485,Aberdeen,2011-02-22 00:00:00,NA,153,146
8485,Aberdeen,2012-02-22 00:00:00,NA,170,140


In [37]:
teammerge[is.na(teammerge)]<-0

In [38]:
head(teammerge)

team_api_id,team_long_name,date,build_up_play,chance_creation,defence
9930,FC Aarau,2010-02-22 00:00:00,0,180,150
9930,FC Aarau,2014-09-19 00:00:00,156,181,145
9930,FC Aarau,2015-09-10 00:00:00,142,181,145
8485,Aberdeen,2010-02-22 00:00:00,0,210,200
8485,Aberdeen,2011-02-22 00:00:00,0,153,146
8485,Aberdeen,2012-02-22 00:00:00,0,170,140


In [39]:
goals<-dbGetQuery(dbcon, "Select home_team_api_id, away_team_api_id,home_team_goal, away_team_goal from Match")

In [40]:
head(goals)

home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
9987,9993,1,1
10000,9994,0,0
9984,8635,0,3
9991,9998,5,0
7947,9985,1,3
8203,8342,1,1


In [41]:
hometeamgoals<-sqldf("Select (sum(home_team_goal)) as homegoals, home_team_api_id from goals group by home_team_api_id")

Loading required package: tcltk
Warning message:
“Quoted identifiers should have class SQL, use DBI::SQL() if the caller performs the quoting.”

In [42]:
head(hometeamgoals)

homegoals,home_team_api_id
148,1601
74,1773
176,1957
70,2033
215,2182
96,2183


In [43]:
awayteamgoals<-sqldf("Select sum((away_team_goal)) as awaygoals, away_team_api_id from goals group by away_team_api_id")

In [44]:
head(awayteamgoals)

awaygoals,away_team_api_id
132,1601
52,1773
111,1957
68,2033
157,2182
88,2183


In [47]:
teamgoals<-sqldf("select (homegoals+awaygoals) as totalteamgoals,away_team_api_id as team_id from hometeamgoals inner join awayteamgoals where home_team_api_id=away_team_api_id")

In [48]:
head(teamgoals)

totalteamgoals,team_id
280,1601
126,1773
287,1957
138,2033
372,2182
184,2183


In [49]:
teammerge<-merge(x=teammerge, y=teamgoals, by.x="team_api_id", by.y="team_id")

In [50]:
head(teammerge)

team_api_id,team_long_name,date,build_up_play,chance_creation,defence,totalteamgoals
1601,Ruch Chorzów,2010-02-22 00:00:00,0,155,175,280
1601,Ruch Chorzów,2011-02-22 00:00:00,0,186,144,280
1601,Ruch Chorzów,2012-02-22 00:00:00,0,159,136,280
1601,Ruch Chorzów,2013-09-20 00:00:00,0,159,136,280
1601,Ruch Chorzów,2014-09-19 00:00:00,139,181,136,280
1601,Ruch Chorzów,2015-09-10 00:00:00,133,176,136,280


## Write the obtained dataframes to csv

In [51]:
write.csv(file="players.csv",playermerge)
write.csv(file="teams.csv",teammerge)

# References:
#### https://stat.ethz.ch/R-manual/R-devel/library/base/html/merge.html
#### https://cran.r-project.org/web/packages/dplyr/vignettes/introduction.html
#### https://cran.r-project.org/web/packages/dplyr/vignettes/two-table.html
#### https://cran.r-project.org/web/packages/RSQLite/RSQLite.pdf